### Metrics, Validation Strategies and Baselines

In [202]:
import pandas as pd
import numpy as np

from itertools import islice, cycle
from more_itertools import pairwise

In [191]:
cd D:\GIT_REPOES\Courses-Cheat_Sheets\Courses\Your First RecSys [ODS]\data

D:\GIT_REPOES\Courses-Cheat_Sheets\Courses\Your First RecSys [ODS]\data


In [192]:
iteractions_df = pd.read_csv('interactions.csv', parse_dates=['start_date'])
users_df = pd.read_csv('users.csv')
items_df = pd.read_csv('items.csv')

### Data Preprocessing
`iteractions_df` DataFrame preprocessing

**Interaction** - `user_id-item_id` log

In [99]:
iteractions_df.head()

,user_id,item_id,progress,rating,start_date
0,126706,14433,80,NaN,2018-01-01
1,127290,140952,58,NaN,2018-01-01
2,66991,198453,89,NaN,2018-01-01
3,46791,83486,23,5.0,2018-01-01
4,79313,188770,88,5.0,2018-01-01


In [80]:
duplicates = iteractions_df.duplicated(subset=['user_id', 'item_id'], keep=False)
df_duplicates = iteractions_df[duplicates].sort_values(by=['user_id', 'start_date'])

# Remove duplicates from iteractions DataFrame
iteractions_df = iteractions_df[~duplicates]

# Fix duplicates DataFrame
df_duplicates = df_duplicates.groupby(['user_id', 'item_id']).agg({
    'progress': 'max',
    'rating': 'max',
    'start_date': 'min'
})

iteractions_df = iteractions_df.append(df_duplicates.reset_index(), ignore_index=True)

# Change Data types 
iteractions_df['progress'] = iteractions_df['progress'].astype(np.int8)
iteractions_df['rating'] = iteractions_df['rating'].astype(pd.SparseDtype(np.float32, np.nan))

`users_df` DataFrame preprocessing

In [84]:
users_df.head()

,user_id,age,sex
0,1,45_54,NaN
1,2,18_24,0.0
2,3,65_inf,0.0
3,4,18_24,0.0
4,5,35_44,0.0


In [85]:
users_df['age'] = users_df['age'].astype('category')
users_df['sex'] = users_df['sex'].astype(pd.SparseDtype(np.float32, np.nan))

In [100]:
# N Unique users in Interactions and Features groups 
unique_users_interactions = set(iteractions_df['user_id'])
unique_users_features = set(users_df['user_id'])

# Split users 
users_interactions_features = unique_users_interactions.intersection(unique_users_features)
users_only_interactions = unique_users_interactions.difference(unique_users_features)
users_only_features = unique_users_features.difference(unique_users_interactions)

n_total_users = len(users_interactions_features) + len(users_only_interactions) + len(users_only_features)

In [104]:
# Statistics for users
print('N Unique Users: ', n_total_users)
print(f'Users Fraction (Only Intercations): {len(unique_users_only_interactions)/n_total_users:.2f}') 
print(f'Users Fraction (Only Features): {len(unique_users_only_features)/n_total_users:.2f}')
print(f'Users Fraction (Intercations and Features): {len(unique_users_interactions_features)/n_total_users:.2f}') 

N Unique Users:  158811
Users Fraction (Only Intercations): 0.10
Users Fraction (Only Features): 0.05
Users Fraction (Intercations and Features): 0.85


`items` DataFrame preprocessing

In [105]:
items_df.head()

,id,title,genres,authors,year
0,128115,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,210979,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,95632,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869
3,247906,Странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,294280,Господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873


In [106]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59599 entries, 0 to 59598
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       59599 non-null  int64 
 1   title    59599 non-null  object
 2   genres   59568 non-null  object
 3   authors  52714 non-null  object
 4   year     46720 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.3+ MB


In [107]:
# Year feature has object....Why???
items_df['year'].value_counts().tail(20)

1924-1941           1
IX в. до н.э.       1
1667                1
1976, 2017          1
1889-1899           1
1995, 1999, 2004    1
2004, 2005          1
1878, 2018          1
2003, 2006, 2011    1
1967, 1969, 1982    1
1887,1902           1
1926,1967           1
1520                1
1937, 1938          1
1597                1
1967, 1968          1
1973, 1975          1
1000                1
1804                1
1931, 1929, 1942    1
Name: year, dtype: int64

In [108]:
# Convert columns to Category type
for col in ['genres', 'authors', 'year']:
    items_df[col] = items_df[col].astype('category')

In [120]:
unique_items_interactions = set(iteractions_df['item_id'])
unique_items_features = set(items_df['id'])

# Split items
items_interactions_features = unique_items_interactions.intersection(unique_items_features)
items_only_interactions = unique_items_interactions.difference(unique_items_features)
items_only_features = unique_items_features.difference(unique_items_interactions)

n_total_items = len(items_interactions_features) + len(items_only_interactions) + len(items_only_features)

In [121]:
# Statistics for items
print('N Unique Items: ', n_total_items)
print(f'Items Fraction (Only Intercations): {len(items_only_interactions)/n_total_items:.2f}') 
print(f'Items Fraction (Only Features): {len(items_only_features)/n_total_items:.2f}')
print(f'items Fraction (Intercations and Features): {len(items_interactions_features)/n_total_items:.2f}') 

N Unique Items:  59599
Items Fraction (Only Intercations): 0.00
Items Fraction (Only Features): 0.00
items Fraction (Intercations and Features): 1.00


### Key Questions to Answer

- What Do We Want from the Model
    - Target value (rating/probability)
    - Items Ranking (Films ranking)
     
- How a Model is Going to be Used?
    - Batch retraining each day
    - Online recommendations
    
- Hardware Limitations
    - The less computational power have, the simplier model is 

- Task Peculiarities
    - Can recommend items that have already been seen by a user?
    - Cold-start problem. How many new users or items do we expect?
    - How many interactions we have 

### Metrics
How a predictions from a recommended system can be described:
- `user_id item_id value/probability`
- `user_id item_id rank/position`

The following types of metrics can be applied
- Regression (MAE, RMSE ...)
- Classification (Precision@K, Recall@K, ...)
- Ranking (DCG@K, NDCG@K, ...)

### Regression Metrics
These metrics describe how well a model predicts a continuous value but not how good the recommendation are

In [127]:
# let's consider an example
df_test = pd.DataFrame({
    'user_id': ['Max', 'Kate', 'Sasha', 'Max'],
    'item_id': ['T-Shirt', 'Dress', 'Jeans', 'Watches'],
    'value': [4, 5, 3, 5]
})

df_preds = pd.DataFrame({
    'user_id': ['Max', 'Kate', 'Sasha', 'Max'],
    'item_id': ['T-Shirt', 'Dress', 'Jeans', 'Watches'],
    'value': [3.28, 3.5, 4.06, 4.73]
})

df_test = df_test.set_index(['user_id', 'item_id'])
df_preds = df_preds.set_index(['user_id', 'item_id'])

In [129]:
df_merged = df_test.join(df_preds, how='left', lsuffix='_true', rsuffix='_recs')
df_merged

,,value_true,value_recs
user_id,item_id,,
Max,T-Shirt,4,3.28
Kate,Dress,5,3.50
Sasha,Jeans,3,4.06
Max,Watches,5,4.73


In [130]:
df_merged['MAE'] = (df_merged['value_true'] - df_merged['value_recs']).abs()
df_merged['MSE'] = (df_merged['value_true'] - df_merged['value_recs']) ** 2
df_merged

,,value_true,value_recs,MAE,MSE
user_id,item_id,,,,
Max,T-Shirt,4,3.28,0.72,0.5184
Kate,Dress,5,3.50,1.50,2.2500
Sasha,Jeans,3,4.06,1.06,1.1236
Max,Watches,5,4.73,0.27,0.0729


### Classification Metrics
Allows estimating **top-k recommendations**

The most importan values - TP and TN
- `True Positive`: Model recommended an object that has been seen/used by a user
- `False Negative`: Model didn't recommend an object that user saw 

We don't take `False Positive` and `True Negative` into  account. In practie, they are very large and we simply neglect them 

In [163]:
# let's consider an example
df_test = pd.DataFrame({
    'user_id': ['Max', 'Kate', 'Sasha', 'Max'],
    'item_id': ['T-Shirt', 'Dress', 'Jeans', 'Watches']
})

df_preds = pd.DataFrame({
    'user_id': [
                'Max', 'Max', 'Max', 
                'Kate', 'Kate', 'Kate',
                'Sasha', 'Sasha', 'Sasha'
               ],
    
    'item_id': [
        'Glasses', 'T-Shirt', 'Socks',
        'T-Shirt', 'Dress', 'Jeans', 
        'Watches', 'Laptop', 'iPods'
                ],
    'rank':[
        1, 3, 2,
        1, 2, 3,
        1, 2, 3
    ]
})

In [164]:
df_test.set_index(['user_id', 'item_id'], inplace=True)
df_preds.set_index(['user_id', 'item_id'], inplace=True)


df_merged = df_test.join(df_preds, how='left')
df_merged

,,rank
user_id,item_id,
Max,T-Shirt,3.0
Kate,Dress,2.0
Sasha,Jeans,NaN
Max,Watches,NaN


### Consider top-2 (K=2)
- Objects that haven't been recommended - ignore (NaN)
- Define: what objects are in top-2
- True Positive for each object
- Calculate `Precision@K` for each user, then average across all users

In [165]:
df_merged['hit@2'] = df_merged['rank'] <= 2
df_merged

,,rank,hit@2
user_id,item_id,,
Max,T-Shirt,3.0,False
Kate,Dress,2.0,True
Sasha,Jeans,NaN,False
Max,Watches,NaN,False


In [166]:
df_merged['hit@2/2'] = df_merged['hit@2'] / 2
df_merged

,,rank,hit@2,hit@2/2
user_id,item_id,,,
Max,T-Shirt,3.0,False,0.0
Kate,Dress,2.0,True,0.5
Sasha,Jeans,NaN,False,0.0
Max,Watches,NaN,False,0.0


In [167]:
# Because use multiindex - use level = 0 (group by user_id)
df_prec_2 = df_merged.groupby(level='user_id')['hit@2/2'].sum()
df_prec_2

user_id
Kate     0.5
Max      0.0
Sasha    0.0
Name: hit@2/2, dtype: float64

In [168]:
print(f'Precision@2: {df_prec_2.mean():.2f}', )

Precision@2: 0.17


In [172]:
# Now, let's calculate for different K
k_set = np.arange(1, 4)
for k in k_set:
    df_merged[f'hit@{k}'] = df_merged['rank'] <= k
    df_merged[f'hit@{k}/{k}'] = df_merged[f'hit@{k}'] / k
    print(f'Precision@{k}: ', df_merged.groupby(level=0)[f'hit@{k}/{k}'].sum().mean())

Precision@1:  0.0
Precision@2:  0.16666666666666666
Precision@3:  0.2222222222222222


In [173]:
df_merged

,,rank,hit@2,hit@2/2,hit@1,hit@1/1,hit@3,hit@3/3
user_id,item_id,,,,,,,
Max,T-Shirt,3.0,False,0.0,False,0.0,True,0.333333
Kate,Dress,2.0,True,0.5,False,0.0,True,0.333333
Sasha,Jeans,NaN,False,0.0,False,0.0,False,0.000000
Max,Watches,NaN,False,0.0,False,0.0,False,0.000000


Similarly calculate `Recall@K` - calculate `hit@K` and devide by the number of relevant objects for a user

In [178]:
unique_users_count = df_merged.index.get_level_values('user_id').nunique()

df_merged['relevant_user_item_count'] = df_merged.groupby(level='user_id')['rank'].transform(np.size)
df_merged['relevant_user_item_count'] 

user_id  item_id
Max      T-Shirt    2.0
Kate     Dress      1.0
Sasha    Jeans      1.0
Max      Watches    2.0
Name: relevant_user_item_count, dtype: float64

In [179]:
for k in k_set:
    print(f'Recall@{k}: ', (df_merged[f'hit@{k}'] / df_merged['relevant_user_item_count']).sum() / unique_users_count)

Recall@1:  0.0
Recall@2:  0.3333333333333333
Recall@3:  0.5


### Ranking Metrics

In [182]:
df_test = pd.DataFrame({
    'user_id': ['Аня', 'Боря', 'Вася', 'Вася'],
    'item_id': ['Мастер и Маргарита', '451° по Фаренгейту', 'Зеленая миля', 'Рита Хейуорт и спасение из Шоушенка'],
})

df_preds = pd.DataFrame({
    'user_id': [
        'Аня', 'Аня', 'Аня', 
        'Боря', 'Боря', 'Боря', 
        'Вася', 'Вася', 'Вася',
    ],
    'item_id': [
        'Отверженные', 'Двенадцать стульев', 'Герои нашего времени', 
        '451° по Фаренгейту', '1984', 'О дивный новый мир',
        'Десять негритят', 'Рита Хейуорт и спасение из Шоушенка', 'Зеленая миля', 
    ],
    'rank': [
        1, 2, 3,
        1, 2, 3,
        1, 2, 3,
    ]
})

In [183]:
df_test.set_index(['user_id', 'item_id'], inplace=True)
df_preds.set_index(['user_id', 'item_id'], inplace=True)


df_merged = df_test.join(df_preds, how='left')
df_merged

rank
user_id item_id                                  
Аня     Мастер и Маргарита                    NaN
Боря    451° по Фаренгейту                    1.0
Вася    Зеленая миля                          3.0
        Рита Хейуорт и спасение из Шоушенка   2.0

In [184]:
# Reciprocal Rank
df_merged['reciprocal_rank'] = 1 / df_merged['rank']
df_merged

rank  reciprocal_rank
user_id item_id                                                   
Аня     Мастер и Маргарита                    NaN              NaN
Боря    451° по Фаренгейту                    1.0         1.000000
Вася    Зеленая миля                          3.0         0.333333
        Рита Хейуорт и спасение из Шоушенка   2.0         0.500000

In [185]:
mrr = df_merged.groupby(level='user_id')['reciprocal_rank'].max()
mrr

user_id
Аня     NaN
Боря    1.0
Вася    0.5
Name: reciprocal_rank, dtype: float64

In [186]:
print(f"MRR = {mrr.fillna(0).mean()}")

MRR = 0.5


In [187]:
df_merged['cumulative_rank'] = df_merged.groupby(level='user_id').cumcount() + 1
df_merged['cumulative_rank'] = df_merged['cumulative_rank'] / df_merged['rank']
df_merged['users_item_count'] = df_merged.groupby(level='user_id')['rank'].transform(np.size)
df_merged

rank  reciprocal_rank  \
user_id item_id                                                      
Аня     Мастер и Маргарита                    NaN              NaN   
Боря    451° по Фаренгейту                    1.0         1.000000   
Вася    Зеленая миля                          3.0         0.333333   
        Рита Хейуорт и спасение из Шоушенка   2.0         0.500000   

                                             cumulative_rank  users_item_count  
user_id item_id                                                                 
Аня     Мастер и Маргарита                               NaN               1.0  
Боря    451° по Фаренгейту                          1.000000               1.0  
Вася    Зеленая миля                                0.333333               2.0  
        Рита Хейуорт и спасение из Шоушенка         1.000000               2.0

In [188]:
users_count = df_merged.index.get_level_values('user_id').nunique()
map3 = (df_merged["cumulative_rank"] / df_merged["users_item_count"]).sum() / users_count
print(f"MAP@3 = {map3}")

MAP@3 = 0.5555555555555556


### Validation (Time Split)


In [199]:
# Select 7 last days for testing 
test_dates = iteractions_df['start_date'].unique()[-7:]
test_dates

array(['2019-12-25T00:00:00.000000000', '2019-12-26T00:00:00.000000000',
       '2019-12-27T00:00:00.000000000', '2019-12-28T00:00:00.000000000',
       '2019-12-29T00:00:00.000000000', '2019-12-30T00:00:00.000000000',
       '2019-12-31T00:00:00.000000000'], dtype='datetime64[ns]')

In [205]:
# Create pairwise dates
test_dates = list(pairwise(test_dates))
test_dates

[(numpy.datetime64('2019-12-25T00:00:00.000000000'),
  numpy.datetime64('2019-12-26T00:00:00.000000000')),
 (numpy.datetime64('2019-12-26T00:00:00.000000000'),
  numpy.datetime64('2019-12-27T00:00:00.000000000')),
 (numpy.datetime64('2019-12-27T00:00:00.000000000'),
  numpy.datetime64('2019-12-28T00:00:00.000000000')),
 (numpy.datetime64('2019-12-28T00:00:00.000000000'),
  numpy.datetime64('2019-12-29T00:00:00.000000000')),
 (numpy.datetime64('2019-12-29T00:00:00.000000000'),
  numpy.datetime64('2019-12-30T00:00:00.000000000')),
 (numpy.datetime64('2019-12-30T00:00:00.000000000'),
  numpy.datetime64('2019-12-31T00:00:00.000000000'))]

In [220]:
# Split the dates
split_dates = test_dates[0]
train_data = iteractions_df[iteractions_df['start_date'] < split_dates[0]]

test_data = iteractions_df[(iteractions_df['start_date'] >= split_dates[0]) & (iteractions_df['start_date'] < split_dates[1])]
test_data = test_data[(test_data['rating'] >= 4) | (test_data['rating'].isnull())]

In [223]:
split_dates, train_data.shape, test_data.shape

((numpy.datetime64('2019-12-25T00:00:00.000000000'),
  numpy.datetime64('2019-12-26T00:00:00.000000000')),
 (1518072, 5),
 (2115, 5))

### Baseline (Recommend Popular)

In [230]:
class PopularRecommender():
    def __init__(self, max_K=100, days=30, item_column='item_id', date_column='date'):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.date_column = date_column
        self.recommendations = []
       
    def fit(self, df):
        min_date = df[self.date_column].max().normalize() - pd.DateOffset(days=self.days)
        self.recommendations = df.loc[df[self.date_column] > min_date, self.item_column].value_counts().head(self.max_K).index.values
        
    def recommend(self, users=None, n=10):
        recomends = self.recommendations[:n]
        if users is None:
            return recomends
        else:
            return list(islice(cycle([recomends]), len(users)))

In [231]:
pop_model = PopularRecommender(days=7, date_column='start_date')
pop_model.fit(train_data)

In [232]:
top10_recs = pop_model.recommend()
top10_recs

array([235407, 230067,  35265, 281005, 147734, 208935, 285394,  96052,
        62715, 151190], dtype=int64)

In [233]:
item_titles = pd.Series(items_df['title'].values, index=items_df['id']).to_dict()
item_titles[128115]

'Ворон-челобитчик'

In [236]:
recs = pd.DataFrame({'user_id': test_data['user_id'].unique()})
top_N = 10
recs['item_id'] = pop_model.recommend(recs['user_id'], n=top_N)
recs.head()

,user_id,item_id
0,38753,"[235407, 230067, 35265, 281005, 147734, 208935..."
1,101642,"[235407, 230067, 35265, 281005, 147734, 208935..."
2,13548,"[235407, 230067, 35265, 281005, 147734, 208935..."
3,130425,"[235407, 230067, 35265, 281005, 147734, 208935..."
4,93986,"[235407, 230067, 35265, 281005, 147734, 208935..."


In [237]:
recs = recs.explode('item_id')
recs.head(top_N + 2)

,user_id,item_id
0,38753,235407
0,38753,230067
0,38753,35265
0,38753,281005
0,38753,147734
0,38753,208935
0,38753,285394
0,38753,96052
0,38753,62715
0,38753,151190


In [238]:
recs['rank'] = recs.groupby('user_id').cumcount() + 1
recs.head(top_N + 2)

,user_id,item_id,rank
0,38753,235407,1
0,38753,230067,2
0,38753,35265,3
0,38753,281005,4
0,38753,147734,5
0,38753,208935,6
0,38753,285394,7
0,38753,96052,8
0,38753,62715,9
0,38753,151190,10


In [239]:
test_recs = test_data.set_index(['user_id', 'item_id']).join(recs.set_index(['user_id', 'item_id']))
test_recs = test_recs.sort_values(by=['user_id', 'rank'])
test_recs.tail()

,,progress,rating,start_date,rank
user_id,item_id,,,,
159294,110617,18,NaN,2019-12-25,NaN
159303,197759,64,NaN,2019-12-25,NaN
159466,124115,84,NaN,2019-12-25,NaN
159472,245992,78,NaN,2019-12-25,NaN
159511,31539,77,5.0,2019-12-25,NaN


In [240]:
test_recs['users_item_count'] = test_recs.groupby(level='user_id', sort=False)['rank'].transform(np.size)
test_recs['reciprocal_rank'] = 1 / test_recs['rank']
test_recs['reciprocal_rank'] = test_recs['reciprocal_rank'].fillna(0)
test_recs['cumulative_rank'] = test_recs.groupby(level='user_id').cumcount() + 1
test_recs['cumulative_rank'] = test_recs['cumulative_rank'] / test_recs['rank']
test_recs.tail()

,,progress,rating,start_date,rank,users_item_count,reciprocal_rank,cumulative_rank
user_id,item_id,,,,,,,
159294,110617,18,NaN,2019-12-25,NaN,1.0,0.0,NaN
159303,197759,64,NaN,2019-12-25,NaN,1.0,0.0,NaN
159466,124115,84,NaN,2019-12-25,NaN,1.0,0.0,NaN
159472,245992,78,NaN,2019-12-25,NaN,1.0,0.0,NaN
159511,31539,77,5.0,2019-12-25,NaN,1.0,0.0,NaN


In [242]:
test_recs[test_recs['rank'].notnull()].head()

,,progress,rating,start_date,rank,users_item_count,reciprocal_rank,cumulative_rank
user_id,item_id,,,,,,,
474,235407,100,5.0,2019-12-25,1.0,1.0,1.000000,1.000000
1672,230067,12,NaN,2019-12-25,2.0,1.0,0.500000,0.500000
2345,208935,76,NaN,2019-12-25,6.0,1.0,0.166667,0.166667
9279,96052,100,NaN,2019-12-25,8.0,1.0,0.125000,0.125000
10260,35265,0,NaN,2019-12-25,3.0,1.0,0.333333,0.333333


In [243]:
print(f'Метрик по test ({str(split_dates[0])[:10]}, {str(split_dates[1])[:10]})')
users_count = test_recs.index.get_level_values('user_id').nunique()
for k in range(1, top_N + 1):
    hit_k = f'hit@{k}'
    test_recs[hit_k] = test_recs['rank'] <= k
    print(f'Precision@{k} = {(test_recs[hit_k] / k).sum() / users_count:.4f}')
    print(f"Recall@{k} = {(test_recs[hit_k] / test_recs['users_item_count']).sum() / users_count:.4f}")

mapN = (test_recs["cumulative_rank"] / test_recs["users_item_count"]).sum() / users_count
print(f"MAP@{top_N} = {mapN}")

mrr = test_recs.groupby(level='user_id')['reciprocal_rank'].max().mean()
print(f"MRR = {mrr}")

Метрик по test (2019-12-25, 2019-12-26)
Precision@1 = 0.0091
Recall@1 = 0.0089
Precision@2 = 0.0071
Recall@2 = 0.0128
Precision@3 = 0.0067
Recall@3 = 0.0173
Precision@4 = 0.0053
Recall@4 = 0.0182
Precision@5 = 0.0047
Recall@5 = 0.0200
Precision@6 = 0.0046
Recall@6 = 0.0240
Precision@7 = 0.0046
Recall@7 = 0.0277
Precision@8 = 0.0042
Recall@8 = 0.0295
Precision@9 = 0.0038
Recall@9 = 0.0297
Precision@10 = 0.0037
Recall@10 = 0.0316
MAP@10 = 0.014664636080969606
MRR = 0.015906783425344247
